In [1]:
import sklearn
import pandas as pd
import numpy as np

from tensorflow import keras
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.regularizers import l2

In [2]:
COMMENT = 'comment_text'
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [3]:
base_path = 'jigsaw-toxic-comment-classification-challenge'
train = pd.read_csv(f'{base_path}/train.csv')
test = pd.read_csv(f'{base_path}/test.csv')
submission = pd.read_csv(f'{base_path}/sample_submission.csv')
test_labels = pd.read_csv(f'{base_path}/test_labels.csv')

In [4]:
EMBEDDING_FILE = f'{base_path}/glove.6B.50d.txt'
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

with open(EMBEDDING_FILE) as f:
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in f)

all_embs = np.stack(embeddings_index.values())
emb_mean = all_embs.mean()
emb_std = all_embs.std()
emb_mean,emb_std

/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(0.020940498, 0.6441043)

# Problem understanding

# Dataset generation

# Metrics define

# Validation strategy

# Data processing (extract useful information)

In [5]:
MAX_TOKENS = 30000
MAX_SENTENSE_LEN = 100
EMBEDDING_SIZE = all_embs.shape[1]

In [6]:
x_train = train["comment_text"].values
y_train = train[LABELS].values
x_test = test["comment_text"].values

tokenizer = text.Tokenizer(num_words=MAX_TOKENS)
tokenizer.fit_on_texts(list(x_train) + list(x_test)) # is it a proper trick?

x_train, x_test = map(tokenizer.texts_to_sequences, [x_train, x_test])
x_train, x_test = map(lambda x: sequence.pad_sequences(x, maxlen=MAX_SENTENSE_LEN), [x_train, x_test])


In [7]:
word_index = tokenizer.word_index
nb_words = min(MAX_TOKENS, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBEDDING_SIZE))
for word, i in word_index.items():
    if i >= MAX_TOKENS: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

# Data understanding & visualization

# De-noise (no drop data)

# Feature engineering

# Offline augmentation

# Standarization

# Scaling

# Normalization

# Feature selection

# Data selection

# Optimization

In [9]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, mask_zero=True, weights=[embedding_matrix], embeddings_regularizer=l2(1e-5))(model_input)
x = keras.layers.GRU(64, return_sequences=True, activation="relu", kernel_regularizer=l2(1e-5))(x)
x = keras.layers.SpatialDropout1D(0.2)(x)
x = keras.layers.GRU(32, return_sequences=False, activation="relu", kernel_regularizer=l2(1e-5))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('71.submission.csv', index=False)

# ~ 0.9629

Epoch 1/100


/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


593/593 [==============================] - 153s 258ms/step - loss: 5.6326 - acc: 0.9034 - val_loss: 4.5540 - val_acc: 0.9952
Epoch 2/100
593/593 [==============================] - 152s 256ms/step - loss: 3.7951 - acc: 0.9845 - val_loss: 3.1240 - val_acc: 0.9952
Epoch 3/100
593/593 [==============================] - 152s 256ms/step - loss: 2.6112 - acc: 0.9914 - val_loss: 2.1549 - val_acc: 0.9952
Epoch 4/100
593/593 [==============================] - 149s 252ms/step - loss: 1.8039 - acc: 0.9927 - val_loss: 1.4912 - val_acc: 0.9952
Epoch 5/100
593/593 [==============================] - 166s 280ms/step - loss: 1.2503 - acc: 0.9929 - val_loss: 1.0361 - val_acc: 0.9952
Epoch 6/100
593/593 [==============================] - 161s 272ms/step - loss: 0.8707 - acc: 0.9928 - val_loss: 0.7258 - val_acc: 0.9952
Epoch 7/100
593/593 [==============================] - 163s 275ms/step - loss: 0.6119 - acc: 0.9922 - val_loss: 0.5143 - val_acc: 0.9952
Epoch 8/100
593/593 [==============================] 

In [10]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, mask_zero=True, weights=[embedding_matrix], embeddings_regularizer=l2(1e-5))(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('72.submission.csv', index=False)

# ~ 0.9701

Epoch 1/100


/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


593/593 [==============================] - 298s 502ms/step - loss: 5.6164 - acc: 0.8554 - val_loss: 4.5436 - val_acc: 0.9937
Epoch 2/100
593/593 [==============================] - 288s 486ms/step - loss: 3.7810 - acc: 0.9905 - val_loss: 3.1088 - val_acc: 0.9949
Epoch 3/100
593/593 [==============================] - 272s 459ms/step - loss: 2.5952 - acc: 0.9922 - val_loss: 2.1401 - val_acc: 0.9949
Epoch 4/100
593/593 [==============================] - 282s 476ms/step - loss: 1.7879 - acc: 0.9912 - val_loss: 1.4768 - val_acc: 0.9946
Epoch 5/100
593/593 [==============================] - 284s 480ms/step - loss: 1.2354 - acc: 0.9929 - val_loss: 1.0226 - val_acc: 0.9951
Epoch 6/100
593/593 [==============================] - 285s 480ms/step - loss: 0.8572 - acc: 0.9926 - val_loss: 0.7134 - val_acc: 0.9942
Epoch 7/100
593/593 [==============================] - 285s 481ms/step - loss: 0.5998 - acc: 0.9907 - val_loss: 0.5030 - val_acc: 0.9934
Epoch 8/100
593/593 [==============================] 

In [11]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.1)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('73.submission.csv', index=False)

# ~ 0.9754

Epoch 1/100
593/593 [==============================] - 243s 410ms/step - loss: 0.1103 - acc: 0.8401 - val_loss: 0.0601 - val_acc: 0.9915
Epoch 2/100
593/593 [==============================] - 246s 414ms/step - loss: 0.0572 - acc: 0.9779 - val_loss: 0.0546 - val_acc: 0.9951
Epoch 3/100
593/593 [==============================] - 249s 420ms/step - loss: 0.0521 - acc: 0.9856 - val_loss: 0.0526 - val_acc: 0.9895
Epoch 4/100
593/593 [==============================] - 251s 423ms/step - loss: 0.0487 - acc: 0.9887 - val_loss: 0.0495 - val_acc: 0.9920
Epoch 5/100
593/593 [==============================] - 249s 420ms/step - loss: 0.0460 - acc: 0.9883 - val_loss: 0.0481 - val_acc: 0.9931
Epoch 6/100
593/593 [==============================] - 247s 416ms/step - loss: 0.0439 - acc: 0.9826 - val_loss: 0.0472 - val_acc: 0.9880
Epoch 7/100
593/593 [==============================] - 224s 377ms/step - loss: 0.0421 - acc: 0.9795 - val_loss: 0.0461 - val_acc: 0.9673
Epoch 8/100
593/593 [====================

In [12]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.1)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=32,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('74.submission.csv', index=False)

# ~ 0.97812

Epoch 1/100
4738/4738 [==============================] - 1132s 239ms/step - loss: 0.0665 - acc: 0.9826 - val_loss: 0.0543 - val_acc: 0.9924
Epoch 2/100
4738/4738 [==============================] - 1069s 226ms/step - loss: 0.0478 - acc: 0.9822 - val_loss: 0.0468 - val_acc: 0.9921
Epoch 3/100
4738/4738 [==============================] - 1081s 228ms/step - loss: 0.0430 - acc: 0.9661 - val_loss: 0.0459 - val_acc: 0.9160
Epoch 4/100
4738/4738 [==============================] - 1139s 240ms/step - loss: 0.0400 - acc: 0.9522 - val_loss: 0.0440 - val_acc: 0.9405
Epoch 5/100
4738/4738 [==============================] - 1554s 328ms/step - loss: 0.0377 - acc: 0.9533 - val_loss: 0.0458 - val_acc: 0.9053
Epoch 00005: early stopping


In [13]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN,))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(50, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(6, activation="sigmoid")(x)
model = keras.Model(inputs=model_input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('75.submission.csv', index=False)

# ~ 0.97808

Epoch 1/100
593/593 [==============================] - 155s 262ms/step - loss: 0.0853 - accuracy: 0.9200 - val_loss: 0.0538 - val_accuracy: 0.9952
Epoch 2/100
593/593 [==============================] - 150s 253ms/step - loss: 0.0502 - accuracy: 0.9545 - val_loss: 0.0496 - val_accuracy: 0.9941
Epoch 3/100
593/593 [==============================] - 150s 254ms/step - loss: 0.0452 - accuracy: 0.9637 - val_loss: 0.0460 - val_accuracy: 0.9949
Epoch 4/100
593/593 [==============================] - 149s 251ms/step - loss: 0.0418 - accuracy: 0.9674 - val_loss: 0.0455 - val_accuracy: 0.9945
Epoch 5/100
593/593 [==============================] - 149s 252ms/step - loss: 0.0394 - accuracy: 0.9589 - val_loss: 0.0449 - val_accuracy: 0.9945
Epoch 6/100
593/593 [==============================] - 150s 253ms/step - loss: 0.0371 - accuracy: 0.9551 - val_loss: 0.0462 - val_accuracy: 0.9936
Epoch 00006: early stopping


In [14]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN,))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(50, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(6, activation="sigmoid")(x)
model = keras.Model(inputs=model_input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=100, validation_split=0.1, 
          callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, verbose=1)], 
         )

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('76.submission.csv', index=False)
# ~ 0.97846

Epoch 1/100
4488/4488 [==============================] - 726s 162ms/step - loss: 0.0593 - accuracy: 0.9621 - val_loss: 0.0493 - val_accuracy: 0.9939
Epoch 2/100
4488/4488 [==============================] - 691s 154ms/step - loss: 0.0442 - accuracy: 0.9841 - val_loss: 0.0459 - val_accuracy: 0.9934
Epoch 3/100
4488/4488 [==============================] - 703s 157ms/step - loss: 0.0394 - accuracy: 0.9657 - val_loss: 0.0460 - val_accuracy: 0.9939
Epoch 00003: early stopping


# Parameter tuning

# Online augmentation

# Model selection / blending

# Post-processing

# Evaluation

# Reasoning

# Monitoring